The system is governed by

\begin{equation}
\nabla\cdot \vec{u} = 0,
\end{equation}

\begin{equation}
\partial_t \vec{u}+\vec{u}\cdot\nabla\vec{u} = -\nabla p + \frac{Pr}{Pe}\nabla^2\vec{u} + \frac{4\pi^2 Ri}{R_\rho-1}(T-S)\vec{e}_z,
\end{equation}

\begin{equation}
\partial_t T +\vec{u}\cdot\nabla T - w = \frac{1}{Pe}\nabla^2 T,
\end{equation}

\begin{equation}
\partial_t S +\vec{u}\cdot\nabla S - R_\rho w = \frac{\tau}{Pe}\nabla^2 S.
\end{equation}

In [ ]:
# This code performs two-dimensional DNS using Dedalus3 based on governing equations in Raiko's paper 
# [Thermohaline layering in dynamically and diffusively stable shear flows]
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3

dealias = 3/2           # scaling factor
pi = np.pi

Ri = 1 # Richardson number [= 1 for figure 3]
Rp = 2   # density ratio
Pe = 1e4 # Peclet number
tau = 0.01 # diffusivity ratio

Lx, Lz = 2, 1
Nx, Nz = 128, 64
# Nx, Nz = 256, 128
# Nx, Nz = 768, 384

# Bases
coords = d3.CartesianCoordinates('x','z')
dist = d3.Distributor(coords, dtype=np.float64)
# define the coordinate system
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
zbasis = d3.RealFourier(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)
# define fields
p = dist.Field(name='p', bases=(xbasis,zbasis)) # pressure
u = dist.VectorField(coords, name='u', bases=(xbasis,zbasis)) # velocity
sa = dist.Field(name='sa', bases=(xbasis,zbasis)) # salinity
te = dist.Field(name='te', bases=(xbasis,zbasis)) # temperature
# Substitutions
x, z = dist.local_grids(xbasis, zbasis) # get coordinate arrays in horizontal and vertical directions
ex, ez = coords.unit_vector_fields(dist) # get unit vectors in horizontal and vertical directions
# define vertical velocity component
w = u @ ez

# create constant sub-field for incompressible flow condition's equation
tau_p = dist.Field(name='tau_p') 
# because this term is only a contant added to the equation, we don't need to instantiate it for bases system

grad_te = d3.grad(te) # First-order reduction
grad_sa = d3.grad(sa) # First-order reduction
grad_u = d3.grad(u) # First-order reduction
# First-order form: "lap(f)" becomes "div(grad_f)"
lap_u = d3.div(grad_u)
lap_te = d3.div(grad_te)
lap_sa = d3.div(grad_sa)
# First-order form: "div(A)" becomes "trace(grad_A)"

# Problem
problem = d3.IVP([p, tau_p, u, te, sa], namespace=locals())

# add equations
problem.add_equation("trace(grad_u) + tau_p = 0")
problem.add_equation("integ(p) = 0") # Pressure gauge
problem.add_equation("dt(u) + grad(p) - (Pr/Pe)*lap_u - (4*pi*pi*Ri/(Rp-1))*(te-sa)*ez = - u@grad(u)")
problem.add_equation("dt(te) - (1./Pe)lap_te - w = - u@grad(te)")
problem.add_equation("dt(sa) - (tau/Pe)*lap_sa - w = - u@grad(sa)")

stop_sim_time = 300 # Stopping criteria
timestepper = d3.RK443 # 3rd-order 4-stage DIRK+ERK scheme [Ascher 1997 sec 2.8] https://doi-org.ezproxy.lib.uconn.edu/10.1016/S0168-9274(97)00056-1
# timestepper = d3.RK222

# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time

# define initial state
te.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
# te['g'] *= z * (Lx - z) # Damp noise at walls
# te['g'] += z # Add linear background

sa.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise

# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time
te.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
# te['g'] *= z * (Lx - z) # Damp noise at walls
# te['g'] += z # Add linear background

sa.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise

# Main loop
print('Starting main loop')
while solver.proceed:
    timestep = CFL.compute_timestep()
    solver.step(timestep)
    if (solver.iteration-1) % 250 == 0:
        print('Completed iteration {}, time={:.3f}'.format(solver.iteration, solver.sim_time))
        sa_plot = plt.pcolormesh(np.copy(sa['g']).transpose())
        plt.colorbar(sa_plot) 
        plt.title("t = {:.3f}".format(solver.sim_time))
        # plt.savefig("salinity{:.3f}.png".format(solver.sim_time), dpi=200)
        plt.show()